### 네이버 뉴스 섹션별 TOPIC 크롤링
**정치, 경제, 사회, 생활/문화, IT/과학, 세계**

In [1]:
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# 02_0에서 저장한 섹션별 link 파일 불러오기
# 
df_menu = pd.read_csv('./crawl_data/naver_news_section.csv', index_col = 0)
df_menu.head()

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...


### 경제 섹션을 예시 코드로 작성
- 정치는 구조가 다르고 경제 ~ 세계 섹션까지는 동일한 프레임 

In [3]:
df_menu['link'][2]

'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'

In [5]:
url = df_menu['link'][2]
# res = requests.get(url)
# ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다
# 원격 조정 봇이라고 생각하고 연결 종료 

#urlopen()은 진행 됨
res = urlopen(url)
res.read()

b'\r\n\r\n\r\n\r\n<!DOCTYPE HTML>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="referrer" contents="always">\r\n<meta http-equiv="refresh" content="600" />\r\n<meta name="viewport" content="width=1106" />\r\n\r\n    \r\n    \r\n        \r\n            \r\n                \r\n                    \r\n                    \r\n                    \r\n                    \r\n                        \r\n\t                        \r\n\t\t                    \r\n\t\t                        \r\n\t\t                        \r\n\t\t                        \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                \r\n                    \r\n                    \r\n                \r\n            \r\n            \r\n            \r\n            \r\n            \r\n        \r\n    \r\n   

- 위 코드처럼 자동화 봇으로 보고 연결 끊는 경우 해결방법
    - 서버에 요청시 header를 구성해 추가해 줌 즉, bot이 아님을 증명
    - header 확인
        - 브라우저에서 해당 사이트 접속 시 생성하는 헤더를
        - 개발자도구로 확인(브라우저 정보 : User-Agent)

In [8]:
# 요청(request 객체 사용)
url = df_menu['link'][2]
headers = {'User-Agent' :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
res = requests.get(url, headers =headers)

In [9]:
html = res.text

In [10]:
html

'\r\n\r\n\r\n\r\n<!DOCTYPE HTML>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="referrer" contents="always">\r\n<meta http-equiv="refresh" content="600" />\r\n<meta name="viewport" content="width=1106" />\r\n\r\n    \r\n    \r\n        \r\n            \r\n                \r\n                    \r\n                    \r\n                    \r\n                    \r\n                        \r\n\t                        \r\n\t\t                    \r\n\t\t                        \r\n\t\t                        \r\n\t\t                        \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                \r\n                    \r\n                    \r\n                \r\n            \r\n            \r\n            \r\n            \r\n            \r\n        \r\n    \r\n    

In [11]:
html = urlopen(url)

In [12]:
bs_obj = BeautifulSoup(html,'html.parser')

- 네이버 뉴스 섹션을 확인하면 각 섹션별 topic이 있고 topic중 헤드라인이 노출
- 섹션에서 수집하는 것 보다 토픽페이지로 이동해서 토픽기사 전체를 크롤링 하면 더 많은 기사를 추출 할 수 있음
- 섹션 페이지 에서는 토픽의 title과 link를 크롤링
- 단, 정치 섹션은 구성이 약간 다르므로 수정해서 사용

In [15]:
# 토픽 추출
topic_list = bs_obj.findAll('h2',{'class':"cluster_head_topic"})
len(topic_list)

13

In [16]:
topic_list[0].find('a')
topic_list[0].find('a').text.replace('\n','')
topic_list[0].find('a')['href'] # sub 주소만 추출됨

<a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202304141430_00000001&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=087&amp;aid=0000964641">
<span class="cluster_head_sub_topic">강릉산불 피해복구 성금 3억 </span><span class="cluster_head_sub_topic">한화그룹 </span>
</a>

In [17]:
# 전체 토픽 title과 link 출력
for topic in topic_list :
    a_t = topic.find('a')
    print(a_t.text.replace('\n',''))
    print(a_t['href'])
# 마지막 부분에서 a 태그가 없는 요소 (빈item)가 있어서 에러 발생
# 예외처리 진행 

강릉산불 피해복구 성금 3억 한화그룹 
/main/clusterArticles.naver?id=c_202304141430_00000001&mode=LSD&mid=shm&sid1=101&oid=087&aid=0000964641
민간아파트 분양가 소폭 상승 1년간 11% 올라 
/main/clusterArticles.naver?id=c_202304171450_00000008&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000894182
3월 車수출 65억달러 6년만에 40만 대 회복했다 
/main/clusterArticles.naver?id=c_202304171110_00000014&mode=LSD&mid=shm&sid1=101&oid=001&aid=0013884721
오프라인 NFC 결제 시작 11번가 간편결제 SK페이 
/main/clusterArticles.naver?id=c_202304170830_00000002&mode=LSD&mid=shm&sid1=101&oid=277&aid=0005246209
인천공항서 보잉 화물기 개조한다 인천공항경제권 MRO산업 날갯짓 
/main/clusterArticles.naver?id=c_202304171010_00000016&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011806901
한은  中 리오프닝 효과 부진
/main/clusterArticles.naver?id=c_202304171210_00000002&mode=LSD&mid=shm&sid1=101&oid=421&aid=0006751826
LH 원가 이하 매입 매입임대주택 고가매입 막는다 
/main/clusterArticles.naver?id=c_202304170850_00000037&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002131005
정부 정책 지원 시급 탄소 포집·활용·저장 뒤처진 한국 
/main/clusterArticles.naver?id=c_202304161800_

AttributeError: 'NoneType' object has no attribute 'text'

In [20]:
for topic in topic_list :
        try :
            a_t = topic.find('a')
            print(a_t.text.replace('\n',''))
            print(a_t['href'])
        except : 
            print('에러발생')
            

강릉산불 피해복구 성금 3억 한화그룹 
/main/clusterArticles.naver?id=c_202304141430_00000001&mode=LSD&mid=shm&sid1=101&oid=087&aid=0000964641
민간아파트 분양가 소폭 상승 1년간 11% 올라 
/main/clusterArticles.naver?id=c_202304171450_00000008&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000894182
3월 車수출 65억달러 6년만에 40만 대 회복했다 
/main/clusterArticles.naver?id=c_202304171110_00000014&mode=LSD&mid=shm&sid1=101&oid=001&aid=0013884721
오프라인 NFC 결제 시작 11번가 간편결제 SK페이 
/main/clusterArticles.naver?id=c_202304170830_00000002&mode=LSD&mid=shm&sid1=101&oid=277&aid=0005246209
인천공항서 보잉 화물기 개조한다 인천공항경제권 MRO산업 날갯짓 
/main/clusterArticles.naver?id=c_202304171010_00000016&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011806901
한은  中 리오프닝 효과 부진
/main/clusterArticles.naver?id=c_202304171210_00000002&mode=LSD&mid=shm&sid1=101&oid=421&aid=0006751826
LH 원가 이하 매입 매입임대주택 고가매입 막는다 
/main/clusterArticles.naver?id=c_202304170850_00000037&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002131005
정부 정책 지원 시급 탄소 포집·활용·저장 뒤처진 한국 
/main/clusterArticles.naver?id=c_202304161800_

- base_url : https://news.naver.com
- sub_url  : /main/main.naver?mode=LSD&mid=shm&sid1=101
- 수집시 기본 url 첨부해서 수집

In [21]:
#link에 기본 url 추가
base_url = 'https://news.naver.com'
for topic in topic_list :
        try :
            a_t = topic.find('a')
            print(a_t.text.replace('\n',''))
            print(base_url + a_t['href'])
        except : 
            print('에러발생')

강릉산불 피해복구 성금 3억 한화그룹 
https://news.naver.com/main/clusterArticles.naver?id=c_202304141430_00000001&mode=LSD&mid=shm&sid1=101&oid=087&aid=0000964641
민간아파트 분양가 소폭 상승 1년간 11% 올라 
https://news.naver.com/main/clusterArticles.naver?id=c_202304171450_00000008&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000894182
3월 車수출 65억달러 6년만에 40만 대 회복했다 
https://news.naver.com/main/clusterArticles.naver?id=c_202304171110_00000014&mode=LSD&mid=shm&sid1=101&oid=001&aid=0013884721
오프라인 NFC 결제 시작 11번가 간편결제 SK페이 
https://news.naver.com/main/clusterArticles.naver?id=c_202304170830_00000002&mode=LSD&mid=shm&sid1=101&oid=277&aid=0005246209
인천공항서 보잉 화물기 개조한다 인천공항경제권 MRO산업 날갯짓 
https://news.naver.com/main/clusterArticles.naver?id=c_202304171010_00000016&mode=LSD&mid=shm&sid1=101&oid=003&aid=0011806901
한은  中 리오프닝 효과 부진
https://news.naver.com/main/clusterArticles.naver?id=c_202304171210_00000002&mode=LSD&mid=shm&sid1=101&oid=421&aid=0006751826
LH 원가 이하 매입 매입임대주택 고가매입 막는다 
https://news.naver.com/main/clusterArticles.naver?

### 수집한 data를 DF로 생성 

In [32]:
topic_title = []
topic_link = []

In [33]:
base_url = 'https://news.naver.com'
for topic in topic_list :
        try :
            a_t = topic.find('a')
            topic_title.append(a_t.text.replace('\n',''))
            topic_link.append(base_url + a_t['href'])
        except : 
            print('에러발생')

에러발생


In [34]:
len(topic_title)

12

In [35]:
len(topic_link)

12

In [36]:
topic_df = pd.DataFrame({'contents':topic_title, 'url':topic_link})

In [37]:
topic_df

,contents,url
0,강릉산불 피해복구 성금 3억 한화그룹,https://news.naver.com/main/clusterArticles.na...
1,민간아파트 분양가 소폭 상승 1년간 11% 올라,https://news.naver.com/main/clusterArticles.na...
2,3월 車수출 65억달러 6년만에 40만 대 회복했다,https://news.naver.com/main/clusterArticles.na...
3,오프라인 NFC 결제 시작 11번가 간편결제 SK페이,https://news.naver.com/main/clusterArticles.na...
4,인천공항서 보잉 화물기 개조한다 인천공항경제권 MRO산업 날갯짓,https://news.naver.com/main/clusterArticles.na...
5,한은 中 리오프닝 효과 부진,https://news.naver.com/main/clusterArticles.na...
6,LH 원가 이하 매입 매입임대주택 고가매입 막는다,https://news.naver.com/main/clusterArticles.na...
7,정부 정책 지원 시급 탄소 포집·활용·저장 뒤처진 한국,https://news.naver.com/main/clusterArticles.na...
8,폐휴대폰 수거 캠페인 진행 삼성전자,https://news.naver.com/main/clusterArticles.na...
9,오피스텔 인기 뚝 1분기 전년比 80%나 급감,https://news.naver.com/main/clusterArticles.na...


### 위 내용을 결합해서 함수로 구성
- 다른 섹션에도 적용하기 위해
- 함수명 : get_topic(url)
- 반환 : 섹션별 수집 데이터를 dict로 구성 후 반환

In [49]:
# 함수 구성 - 섹션별 페이지 요청, topic 라인 크롤링 (토픽내용/링크추출)
def get_topic(url,section) :
    res = requests.get(url, headers =headers)
    
    # 파서기 (bs4 객체)
    html = res.text
    bs_obj = BeautifulSoup(html, 'html.parser')
    
    # 토픽추출
    # h2 태그의 class : cluster_head_topic
    topic_list=bs_obj.findAll("h2",{"class":"cluster_head_topic"})
    
    #최종 수집 항목 list에 저장 
    #link에 기본 url 추가
    topic_title = []
    topic_link =  []
    for topic in topic_list : 
        try : 
            a_t = topic.find('a')
            topic_title.append(a_t.text.replace('\n',''))
            topic_link.append(base_url + a_t['href'])
        except :
            print('에러발생')            
    return({'topic' : topic_title, 'url' : topic_link,'section' :section})

In [50]:
# 함수 테스트
url 
get_topic(url,'경제')

에러발생


{'topic': ['오피스텔 인기 뚝 1분기 전년比 80%나 급감 ',
  '"세수 펑크" 우려에 정부 근로장려금·월세공제 구조조정 시험대 ',
  '유류세 조정여부 이번주 결정 추경호 "민생부담 진지하게 고려" ',
  '금융분야 AI 보안지침 나왔다 AI기반 신용평가 신뢰도 높인다 ',
  'LH 원가 이하 매입 매입임대주택 고가매입 막는다 ',
  '폐휴대폰 수거 캠페인 진행 삼성전자 ',
  '"돌돌 말리는 車 디스플레이" 세계 최초 개발 현대모비스 ',
  '증권사 해외법인 대출규제 완화 금융위 종투사 NCR규제 개선 ',
  '1분기 벤처투자 실적 60% 줄어 8815억 ',
  '1분기 ELS 발행 9조7천억원 간만의 증시 훈풍에 ',
  '정부 정책 지원 시급 탄소 포집·활용·저장 뒤처진 한국 ',
  '강릉산불 피해복구 성금 3억 한화그룹 '],
 'url': ['https://news.naver.com/main/clusterArticles.naver?id=c_202304170950_00000004&mode=LSD&mid=shm&sid1=101&oid=008&aid=0004875905',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202304170650_00000037&mode=LSD&mid=shm&sid1=101&oid=014&aid=0004997805',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202304161210_00000009&mode=LSD&mid=shm&sid1=101&oid=056&aid=0011467057',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202304171210_00000003&mode=LSD&mid=shm&sid1=101&oid=001&aid=0013884989',
  'https://news.naver.com/main/clust

In [67]:
# 02_0에서 저장한 섹션별 url 파일 가져오기
df_menu = pd.read_csv('./crawl_data/naver_news_section.csv', index_col=0)
df_menu.head(8)

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...


In [68]:
# 함수테스트
pd.DataFrame(get_topic(df_menu['link'][3],df_menu['section'][3]))

에러발생


,topic,url,section
0,"육아휴직 사용방해 집중감독 고용부 ""모성보호 신고센터"" 개설",https://news.naver.com/main/clusterArticles.na...,사회
1,5일에 한 번꼴로 안전사고 개화∼김포공항 버스전용차로 최단기간 개설,https://news.naver.com/main/clusterArticles.na...,사회
2,"엠폭스 11~13번째 확진자 발생 ""국내 감염"" 추정",https://news.naver.com/main/clusterArticles.na...,사회
3,제주공항에 또 드론 항공기 이착륙 15분간 중단,https://news.naver.com/main/clusterArticles.na...,사회
4,"""대한민국 안전대전환"" 집중 안전 점검실시 노후교량 안전점검 대상에 포함",https://news.naver.com/main/clusterArticles.na...,사회
5,"""민주당 돈봉투"" 의혹 강래구 소환 조사 검찰",https://news.naver.com/main/clusterArticles.na...,사회
6,"""전우원 폭로"" 마약투약 지인 3명 ""검사 결과 음성""",https://news.naver.com/main/clusterArticles.na...,사회
7,"관광객 입도세 추진 ""제주도 들어오려면 8000원""",https://news.naver.com/main/clusterArticles.na...,사회
8,간호법 국회 통과하면 총파업 거리로 나온 의사·간호조무사들,https://news.naver.com/main/clusterArticles.na...,사회
9,음주운전 9명 적발 특별단속 예고에도 아랑곳,https://news.naver.com/main/clusterArticles.na...,사회


In [69]:
# 모든 섹션의 topic을 하나의 dataframe에 저장하기 위해 빈 dataframe을 생성
dict_sub={}
dict_sub['topic']=[]
dict_sub['url']=[]
dict_sub['section']=[]

topic_df = pd.DataFrame(dict_sub)
topic_df

,topic,url,section


In [70]:
df_menu.head(8)

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...


In [71]:
for i in range(1,7) :
    temp = pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df = pd.concat([topic_df,temp],axis=0,ignore_index=True)

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


In [72]:
topic_df.head()
topic_df.tail()

,topic,url,section
49,"스페이스X 스타십 발사 승인 ""17일 비행 목표""",https://news.naver.com/main/clusterArticles.na...,세계
50,"EU 외교수장 ""G7서 북중현안 논의 중요""",https://news.naver.com/main/clusterArticles.na...,세계
51,멜버른 시드니 제쳐 최대 인구 도시 뒤집힌 호주,https://news.naver.com/main/clusterArticles.na...,세계
52,"용의자 묵비권 행사 기시다 日총리 ""폭탄 테러""",https://news.naver.com/main/clusterArticles.na...,세계
53,새 법인 X.AI 설립 머스크 X파일 실체 드러났다,https://news.naver.com/main/clusterArticles.na...,세계


### 위 결과에는 정치 섹션이 들어오지 않음 - 태그 구성이 다르기 때문
- 정치 섹션도 포함되게 함수 변경
- https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100 를 확인하면 topic이 따로 없음

In [76]:
# 함수코드
# 함수 구성 - 섹션별 페이지 요청, topic 라인 크롤링 (토픽내용/링크추출)
def get_topic(url,section) :
    res = requests.get(url, headers =headers)
    
    # 파서기 (bs4 객체)
    html = res.text
    bs_obj = BeautifulSoup(html, 'html.parser')
    
    # 토픽추출
    # section이 정치인 경우는 다른 태그를 추출
    if section == '정치':
        topic_list = bs_obj.findAll('div',{'class':'cluster_foot_inner'})
    else :
        topic_list=bs_obj.findAll("h2",{"class":"cluster_head_topic"})
    
    #최종 수집 항목 list에 저장 
    #link에 기본 url 추가
    topic_title = []
    topic_link =  []
    for topic in topic_list : 
        try : 
            a_t = topic.find('a')
            topic_title.append(a_t.text.replace('\n',''))
            topic_link.append(base_url + a_t['href'])
        except :
            print('에러발생')            
    return({'topic' : topic_title, 'url' : topic_link,'section' :section})

In [77]:
# 모든 섹션의 topic을 하나의 dataframe에 저장하기 위해 빈 dataframe을 생성
dict_sub={}
dict_sub['topic']=[]
dict_sub['url']=[]
dict_sub['section']=[]

topic_df = pd.DataFrame(dict_sub)
topic_df

,topic,url,section


In [78]:
for i in range(1,7) :
    temp = pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df = pd.concat([topic_df,temp],axis=0,ignore_index=True)

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


In [82]:
topic_df.head()
topic_df.tail()

,topic,url,section
65,우크라 곡물 수입 중단 폴란드·헝가리,https://news.naver.com/main/clusterArticles.na...,세계
66,푸틴 연합훈련·전문교류 강화하자 中 국방부장 만나,https://news.naver.com/main/clusterArticles.na...,세계
67,새 법인 X.AI 설립 머스크 X파일 실체 드러났다,https://news.naver.com/main/clusterArticles.na...,세계
68,"""기밀문서 유출"" 청문회 추진 美 공화 ""너무 나갔다""",https://news.naver.com/main/clusterArticles.na...,세계
69,"""中 군사훈련"" 끝나자 ""16일 대만해협으로 美군함 통과""",https://news.naver.com/main/clusterArticles.na...,세계


In [80]:
# 파일로 저장
topic_df.to_csv('./crawl_data/naver_news_topic.csv')